# Advent of Code

## 2021-012-023
## 2021 023

https://adventofcode.com/2021/day/23

In [4]:
import heapq

def parse_input(filename):
    # Expected format (similar to AoC 2021 Day 23):
    #
    # #############
    # #...........#
    # ###C#A#B#D###
    #   #B#A#D#C#
    #   #########
    #
    # We'll parse the input to extract the initial state.
    
    lines = [line.rstrip('\n') for line in open(filename)]
    # Hallway: line 1 (indexing from 0): lines[1] should be "#...........#"
    hallway_line = lines[1][1:12]  # Extract just the hallway positions (.)
    
    # Rooms from lines:
    # The top row of rooms: lines[2]: "###C#A#B#D###"
    # The bottom row of rooms: lines[3]: "  #B#A#D#C#"
    # We know rooms are always in columns at indices:
    # Rooms: positions: 3,5,7,9 for top row in line 2
    #        positions: 3,5,7,9 for bottom row in line 3
    # This is a 2-depth puzzle (part 1 form).
    
    top_row = lines[2]
    bottom_row = lines[3]
    # Extract amphipods in each room:
    roomA = (top_row[3], bottom_row[3]) # leftmost
    roomB = (top_row[5], bottom_row[5])
    roomC = (top_row[7], bottom_row[7])
    roomD = (top_row[9], bottom_row[9])
    
    # Hallway initially empty (.)
    hallway = tuple(hallway_line)
    
    # State format:
    # hallway: tuple of length 11
    # rooms: tuple of 4 tuples, each a room, from left to right A-D.
    # We'll store rooms as tuple of strings representing amphipods from top to bottom.
    return (hallway, (roomA, roomB, roomC, roomD))


# Energy costs per amphipod type
ENERGY = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

# Which room each amphipod wants to go to:
DESTINATION = {'A':0, 'B':1, 'C':2, 'D':3}

# The valid hallway positions (indices 0 through 10)
HALLWAY_LEN = 11
# The positions outside rooms that amphipods cannot stop at permanently:
NO_STOP = {2,4,6,8}

def is_organized(state):
    # Check if each room is filled with the correct amphipods only
    hallway, rooms = state
    # hallway should be empty
    if any(pos != '.' for pos in hallway):
        return False
    # rooms should be:
    # roomA: all 'A', roomB: all 'B', etc.
    desired = ['A','B','C','D']
    for i, room in enumerate(rooms):
        if any(a != desired[i] for a in room):
            return False
    return True

def room_can_accept(rooms, room_index, amph):
    # A room can accept amph if:
    # - All amphipods in the room are either '.' or the same type amph (but we have no '.' in final representation, just less filled)
    # In our model, the rooms always have 2 positions. If not empty, must contain only the target amph type.
    # Since we're storing them as tuples of amphipods, no '.' is stored. They are always occupied. 
    # Actually, for states, we should consider that a room can have top portion accessible:
    # We'll consider partial states as well. Let's assume we allow "empty" representation by allowing shorter tuples if needed.
    #
    # If we adapt the code to handle states that have popped from rooms, we might represent empty spots by '.' as well.
    # For simplicity, let's assume no '.' in the room representation. If a spot is empty, we represent it by None or omit it.
    # Let's store rooms as tuples of length 2. If an amphipod moved out, we replace it with '.' to indicate empty.
    # This means after parsing, if top amph is 'C', bottom 'B', initially they are both filled. After moves, we can have '.'.
    # We'll rewrite initial parsing to store '.' for empties when needed. For now no empties at start, but empties will appear.
    
    # Let's define a helper: a room is always length 2. The bottom elements fill from bottom to top. If top is '.' the room has a vacancy.
    # A room can accept 'A' if all current non-dot amphipods are 'A'.
    # Similarly for B, C, D.
    
    room = rooms[room_index]
    # All must match the desired type if not '.'
    desired = ['A','B','C','D'][room_index]
    return all(x in (desired, '.') for x in room)

def room_depth(rooms, room_index):
    # Count how many amphipods are in the room (not '.')
    return sum(1 for x in rooms[room_index] if x != '.')

def get_room_entry_hallway_index(room_index):
    # Rooms are aligned under hallway positions:
    # roomA -> hallway index 2
    # roomB -> hallway index 4
    # roomC -> hallway index 6
    # roomD -> hallway index 8
    return (room_index+1)*2

def hallway_path_clear(hallway, start, end):
    # Check if hallway between start and end (inclusive of end, exclusive of start maybe) is clear ('.')
    if start < end:
        path = hallway[start+1:end+1]
    else:
        path = hallway[end:start]
    return all(x == '.' for x in path)

def generate_moves(state):
    # Given a state, yield possible next states and the cost to get there.
    hallway, rooms = state

    # Moves from room to hallway:
    # If a room has a top amph to move out (the top non-empty position):
    # Check who is on top. That amph can move out if needed.
    # They can move to any hallway position that is '.' and not outside a room (unless continuing).
    # Actually, they can stop in the hallway if not in NO_STOP positions or if forced. The rules are complex.
    # Let's clarify the rules:
    #
    # According to the puzzle:
    # - Amphipods will never stop on the space immediately outside any room unless they have no choice moving through it.
    #   Actually, they CAN move through but not stop there if they are leaving a room. 
    #   They must stop only in hallway positions not in NO_STOP or in their final room.
    #
    # But the puzzle states they do not stop outside rooms in the hallway. The standard interpretation:
    # - They can pass over these spots, but cannot end there.
    # 
    # For moves from room to hallway:
    # The amphipod moves up out of the room and can move left or right until it hits a wall or another amphipod.
    # They can choose any valid hallway position (except those directly outside rooms) to stop.

    # Identify top movable amphipod in each room:
    for r_i in range(4):
        room = rooms[r_i]
        # Determine which amphipod is accessible from this room:
        # If top position is '.', that means top is empty, maybe bottom is amph. We must move from the topmost occupied.
        # The "top" of the room is room[0], the "bottom" is room[1] in our indexing if we said top then bottom.
        # Actually, let's assume room[0] is the top position and room[1] is the bottom. The puzzle top is first line we parsed.
        # The amphipod that can leave the room is the topmost that isn't '.'.
        # In a 2-depth puzzle: if room[0] != '.' then that is the top amph. else if room[0] == '.' and room[1] != '.' then bottom is accessible now.

        # Actually, the puzzle rooms are vertically arranged:
        # Input format we used: room = (top, bottom)
        # top is room[0], bottom is room[1].
        # If top is '.', then we must check bottom. If bottom != '.' and top='.', then bottom amph can move up first by effectively vacating bottom -> top and then out.
        # But the puzzle states we must move out in open space. Actually, if top is '.' and bottom is 'X', we can move that 'X' out by counting steps = 2 steps to exit the room.
        # Step cost to exit room: If top is '.', we must move bottom amph up 1 step plus out into the hallway. That counts as 2 steps total to get to hallway line.
        
        # Let's find the first non '.' from the top:
        top_pos = None
        depth_count = 0
        for depth, val in enumerate(room):
            if val != '.':
                top_pos = depth
                break

        if top_pos is None:
            # room empty, no amph to move out
            continue

        amph = room[top_pos]
        # If this amph already in correct room and all below it are also correct, it doesn't need to move:
        # Check if this room is the correct room for amph:
        desired_room = DESTINATION[amph]
        if desired_room == r_i:
            # Check if all below are also correct:
            if all(x == amph for x in room[top_pos+1:] if x != '.'):
                # no need to move this one out
                continue

        # Steps to move from inside the room to the hallway "line" (the first row):
        # top amph in top position: 1 step to get out
        # If top amph is at depth=0, 1 step to hallway
        # if at depth=1, 2 steps, etc.
        steps_out_of_room = top_pos + 1

        room_entrance = get_room_entry_hallway_index(r_i)
        # The amph can move left or right in the hallway:
        # Check left side
        for h_i in range(room_entrance-1, -1, -1):
            if hallway[h_i] != '.':
                break
            # can't stop at NO_STOP positions
            if h_i not in NO_STOP:
                # This is a valid stopping position
                dist = steps_out_of_room + (room_entrance - h_i)
                cost = ENERGY[amph] * dist
                new_hallway = list(hallway)
                new_rooms = [list(r) for r in rooms]
                # Move amph out of room
                new_hallway[h_i] = amph
                # Clear that amph from the room
                new_rooms[r_i][top_pos] = '.'
                yield ((tuple(new_hallway), tuple(tuple(x) for x in new_rooms)), cost)

        # Check right side
        for h_i in range(room_entrance+1, HALLWAY_LEN):
            if hallway[h_i] != '.':
                break
            if h_i not in NO_STOP:
                dist = steps_out_of_room + (h_i - room_entrance)
                cost = ENERGY[amph] * dist
                new_hallway = list(hallway)
                new_rooms = [list(r) for r in rooms]
                new_hallway[h_i] = amph
                new_rooms[r_i][top_pos] = '.'
                yield ((tuple(new_hallway), tuple(tuple(x) for x in new_rooms)), cost)


    # Moves from hallway to room:
    # If an amph in the hallway can move directly into its destination room and that room can accept it:
    for h_i, amph in enumerate(hallway):
        if amph == '.':
            continue
        target_room = DESTINATION[amph]
        # Check if room can accept it:
        if room_can_accept(rooms, target_room, amph):
            # The amph can move into that room if path is clear
            room_entrance = get_room_entry_hallway_index(target_room)
            # Check if path is clear between h_i and room_entrance (not including h_i itself)
            if hallway_path_clear(hallway, h_i, room_entrance):
                # Compute cost
                # Steps in hallway: abs(room_entrance - h_i)
                # Steps into room: the room depth - from top
                # The correct position in the room to place amph is the bottommost '.' slot:
                # For a 2-depth room, if room is partially filled, amph goes to the lowest '.'.
                # The movement cost inside room = depth index + 1
                new_room = list(rooms[target_room])
                # Fill from bottom:
                # Find the deepest '.' in the target room
                insert_depth = None
                for d in reversed(range(len(new_room))):
                    if new_room[d] == '.':
                        insert_depth = d
                        break
                # insert_depth should not be None since we checked acceptance.
                dist = abs(room_entrance - h_i) + (insert_depth+1)  # steps out in hallway + steps down into room
                cost = ENERGY[amph] * dist

                new_hallway = list(hallway)
                new_hallway[h_i] = '.'
                new_rooms = [list(r) for r in rooms]
                new_rooms[target_room][insert_depth] = amph

                yield ((tuple(new_hallway), tuple(tuple(x) for x in new_rooms)), cost)

def solve(initial_state):
    # Use Dijkstra to find minimum cost to organized state
    # State: (hallway, (roomA, roomB, roomC, roomD))
    # We'll store visited states with best cost
    import sys
    visited = {}
    pq = []
    heapq.heappush(pq, (0, initial_state))
    while pq:
        cost, state = heapq.heappop(pq)
        if state in visited and visited[state] <= cost:
            continue
        visited[state] = cost
        if is_organized(state):
            return cost
        for nxt, cst in generate_moves(state):
            new_cost = cost + cst
            if nxt not in visited or visited[nxt] > new_cost:
                heapq.heappush(pq, (new_cost, nxt))
    return None

def main():
    initial_state = parse_input("input.txt")
    # We must ensure rooms are mutable as lists or we always convert carefully.
    # Let's convert initial room states to the format we want: Each room is a tuple of length 2. Replace no empty with '.' if needed.
    # The parsed input had full amphipods, no '.' initially, but let's normalize:
    hallway, rooms = initial_state
    # Ensure rooms are tuples of length 2 each:
    # They are already like ('C','B') for example. No '.' initially.
    # But we need '.' later. Let's just trust our code to handle '.' if we create states with them.
    # We'll trust initial is always full, so no '.' needed initially.
    # Just ensure final format:
    rooms = tuple(tuple(x for x in r) for r in rooms)
    # Replace initial_state:
    initial_state = (hallway, rooms)

    # Convert rooms to have '.' if needed. Actually, we must be consistent. The code expects '.' to mark empties.
    # Initially, all rooms are full. No empties. So no change needed now.

    answer = solve(initial_state)
    print(answer)

if __name__ == "__main__":
    main()

11516


In [5]:
import heapq

def parse_input(filename):
    lines = [line.rstrip('\n') for line in open(filename)]

    # Original part 1 lines (2-depth):
    # Example input:
    # #############
    # #...........#
    # ###C#A#B#D###
    #   #B#A#D#C#
    #   #########

    # For part two, we insert two extra lines of amphipods between these two room lines:
    #   #D#C#B#A#
    #   #D#B#A#C#
    #
    # Final structure (4-depth):
    # ###C#A#B#D###
    #   #D#C#B#A#
    #   #D#B#A#C#
    #   #B#A#D#C#

    # Extract hallway from line 1 (0-based): lines[1][1:12]
    hallway_line = lines[1][1:12]
    hallway = tuple(hallway_line)

    # Extract the top and bottom lines of the original input:
    top_rooms_line = lines[2]   # e.g. "###C#A#B#D###"
    bottom_rooms_line = lines[3] # e.g. "  #B#A#D#C#"

    # Insert the two extra lines:
    # For Part 2:
    # Insert after top_rooms_line:
    #   #D#C#B#A#
    #   #D#B#A#C#
    extra_line_1 = "  #D#C#B#A#"
    extra_line_2 = "  #D#B#A#C#"

    # Now we have 4 layers. The order from top to bottom will be:
    # Layer 0 (top): from top_rooms_line
    # Layer 1: from extra_line_1
    # Layer 2: from extra_line_2
    # Layer 3 (bottom): from bottom_rooms_line

    # Rooms are at columns 3,5,7,9 in these lines (same indexing as part 1)
    def get_rooms(line):
        return (line[3], line[5], line[7], line[9])

    layer0 = get_rooms(top_rooms_line)     # e.g. ('C','A','B','D')
    layer1 = get_rooms(extra_line_1)       # ('D','C','B','A')
    layer2 = get_rooms(extra_line_2)       # ('D','B','A','C')
    layer3 = get_rooms(bottom_rooms_line)  # ('B','A','D','C')

    # Rooms: We want each room as a tuple of length 4: topmost is layer0, bottom-most is layer3
    # For room A (leftmost room is index 0 in these tuples):
    # That room's stack: (layer0[0], layer1[0], layer2[0], layer3[0])
    # Do this for each room column

    roomA = (layer0[0], layer1[0], layer2[0], layer3[0])
    roomB = (layer0[1], layer1[1], layer2[1], layer3[1])
    roomC = (layer0[2], layer1[2], layer2[2], layer3[2])
    roomD = (layer0[3], layer1[3], layer2[3], layer3[3])

    return (hallway, (roomA, roomB, roomC, roomD))

# Energy costs per amphipod type
ENERGY = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

# Which room each amphipod wants to go to:
DESTINATION = {'A':0, 'B':1, 'C':2, 'D':3}

HALLWAY_LEN = 11
NO_STOP = {2,4,6,8}  # positions outside each room in the hallway

def is_organized(state):
    hallway, rooms = state
    if any(pos != '.' for pos in hallway):
        return False
    # For part 2: each room must have all four amphipods correct
    desired = ['A','B','C','D']
    for i, room in enumerate(rooms):
        if any(a != desired[i] for a in room):
            return False
    return True

def room_can_accept(rooms, room_index, amph):
    # In part 2, a room has 4 positions.
    # A room can accept 'A' if all non-'.' are 'A', and there is at least one '.' for space.
    desired = ['A','B','C','D'][room_index]
    room = rooms[room_index]
    # All must be desired or '.' and must have at least one '.' to place the amph
    return all(x in (desired, '.') for x in room) and '.' in room

def get_room_entry_hallway_index(room_index):
    return (room_index+1)*2

def hallway_path_clear(hallway, start, end):
    if start < end:
        path = hallway[start+1:end+1]
    else:
        path = hallway[end:start]
    return all(x == '.' for x in path)

def generate_moves(state):
    hallway, rooms = state

    # Moves from room to hallway
    # Find the topmost amphipod in each room (the first from top that is not '.'):
    # If a room is (top,b,c,d) top=0 index, bottom=3
    # The amph to move out is the first non '.' from the top.
    for r_i, room in enumerate(rooms):
        top_pos = None
        for depth, val in enumerate(room):
            if val != '.':
                top_pos = depth
                break
        if top_pos is None:
            # empty room
            continue

        amph = room[top_pos]
        desired_room = DESTINATION[amph]
        # If this amph is already in its correct room and all below it are also correct, no need to move:
        if desired_room == r_i:
            below = room[top_pos+1:]
            if all(x == amph for x in below):
                continue

        steps_out_of_room = top_pos + 1  # depth index + 1 steps to get out

        room_entrance = get_room_entry_hallway_index(r_i)
        # Try moving left
        for h_i in range(room_entrance-1, -1, -1):
            if hallway[h_i] != '.':
                break
            if h_i not in NO_STOP:
                dist = steps_out_of_room + (room_entrance - h_i)
                cost = ENERGY[amph] * dist
                new_hallway = list(hallway)
                new_rooms = [list(r) for r in rooms]
                new_hallway[h_i] = amph
                new_rooms[r_i][top_pos] = '.'
                yield ((tuple(new_hallway), tuple(tuple(x) for x in new_rooms)), cost)

        # Try moving right
        for h_i in range(room_entrance+1, HALLWAY_LEN):
            if hallway[h_i] != '.':
                break
            if h_i not in NO_STOP:
                dist = steps_out_of_room + (h_i - room_entrance)
                cost = ENERGY[amph] * dist
                new_hallway = list(hallway)
                new_rooms = [list(r) for r in rooms]
                new_hallway[h_i] = amph
                new_rooms[r_i][top_pos] = '.'
                yield ((tuple(new_hallway), tuple(tuple(x) for x in new_rooms)), cost)

    # Moves from hallway to room
    for h_i, amph in enumerate(hallway):
        if amph == '.':
            continue
        target_room = DESTINATION[amph]
        if room_can_accept(rooms, target_room, amph):
            room_entrance = get_room_entry_hallway_index(target_room)
            if hallway_path_clear(hallway, h_i, room_entrance):
                # Find the deepest '.' in the target room
                new_room = list(rooms[target_room])
                insert_depth = None
                for d in reversed(range(len(new_room))):
                    if new_room[d] == '.':
                        insert_depth = d
                        break
                dist = abs(room_entrance - h_i) + (insert_depth+1)
                cost = ENERGY[amph] * dist

                new_hallway = list(hallway)
                new_hallway[h_i] = '.'
                new_rooms = [list(r) for r in rooms]
                new_rooms[target_room][insert_depth] = amph

                yield ((tuple(new_hallway), tuple(tuple(x) for x in new_rooms)), cost)


def solve(initial_state):
    visited = {}
    pq = []
    heapq.heappush(pq, (0, initial_state))
    while pq:
        cost, state = heapq.heappop(pq)
        if state in visited and visited[state] <= cost:
            continue
        visited[state] = cost
        if is_organized(state):
            return cost
        for nxt, cst in generate_moves(state):
            new_cost = cost + cst
            if nxt not in visited or visited[nxt] > new_cost:
                heapq.heappush(pq, (new_cost, nxt))
    return None

def main():
    initial_state = parse_input("input.txt")

    # Convert rooms to a format with '.' for empty spaces if needed
    # Initially all spots are filled, no '.' needed at start. 
    # But we will rely on '.' replacements during moves.
    hallway, rooms = initial_state
    # Ensure rooms are all tuples of length 4:
    # Already 4-depth from parsing.
    # Replace is not needed. All are amphipods now, no '.'.
    # The code will handle '.' when rooms become partially empty.

    answer = solve((hallway, rooms))
    print(answer)

if __name__ == "__main__":
    main()

40272
